In [1]:
# We will need these so we can reload modules as we modify them
%load_ext autoreload
%autoreload 2

In [3]:
clusterId = 'j-1TFS8W6PX3DIQ'

In [10]:
%%writefile MRJob_ReadFromS3.py
from mrjob.job import MRJob
from mrjob.step import MRStep

class job(MRJob):
        
    # Specify some custom options so we only have to write one MRJob class for each join
    def configure_options(self):
        super(job, self).configure_options()
        self.add_passthrough_option('--endNode', default='1')
        self.add_passthrough_option('--startNode', default='1')
        
    def mapper(self, _, line):

        # Split text to get our data
        fields = line.strip().split('\t')
        
        # If using EMR, need to eval the string
        name = eval(fields[0])
        value = eval(fields[1])
        neighbors = value[0]
        distance = int(value[1])
        status = value[2]
        path = value[3]
        
        if name == self.options.endNode or name == self.options.startNode:
            yield name, [neighbors, distance, status, path]
        
    def reducer(self, key, values):
        for val in values:
            yield key, val
        
if __name__ == '__main__':
    job.run()

Overwriting MRJob_ReadFromS3.py


In [14]:
from MRJob_ReadFromS3 import job

inputDir = 's3://ms-w261-hw07/all-pages-indexed-out_state2/'
outputDir = 's3://ms-w261-hw07/output'
!aws s3 rm --recursive --quiet {outputDir}

mr_job = job(args=[inputDir, '--no-strict-protocols', '--endNode', '5553516', '--startNode', '8375315',
                   '-r', 'emr', '--emr-job-flow-id', clusterId,
                   '--output-dir', outputDir])
output = []

with mr_job.make_runner() as runner: 
    # Run MRJob
    runner.run()

    # Write stream_output to file
    for line in runner.stream_output():
        out = mr_job.parse_output_line(line)
        output.append(out)

print output

[('8375315', [{'3084598': 1, '1861707': 1, '11936758': 2, '13466359': 2, '9817862': 1, '13350692': 3, '615336': 1, '696523': 1, '14839194': 1, '5553516': 4, '13455888': 1, '5553374': 1, '1441162': 1, '8730327': 2}, 0, 'Q', ['8375315']]), ('5553516', [{'5796005': 1, '8460470': 1, '1441772': 4, '5862396': 1, '14851765': 1, '13089808': 1, '2719428': 1, '15135279': 1, '3160031': 1, '5192300': 1, '12739465': 1, '8501386': 1, '7695681': 1, '5553541': 7, '1057671': 1, '11442329': 1, '4631775': 1, '5553268': 1, '5553381': 4, '8862731': 1, '5553260': 1, '10564700': 1, '5661960': 1, '4646934': 1, '14528185': 1, '14614864': 1, '1053553': 1, '10989367': 1, '9861552': 1, '14466248': 1, '14699310': 1, '11590123': 1, '14027756': 1, '3931046': 1, '5553167': 2, '12490541': 1, '14712295': 1, '3958278': 1, '10494054': 1, '11173600': 1, '5653931': 1, '6802150': 1, '5553445': 1, '11460359': 1, '8400764': 1, '10094925': 1, '9415038': 1, '9106952': 1, '9936432': 1, '6303147': 1, '957342': 1, '2416328': 1, '1